# Working with Raster Properties and Transformations

## Understanding raster properties with rioxarray

Rioxarray provides direct access to important raster properties and metadata through its `rio` accessor. Unlike context managers that set default behaviors, rioxarray uses method chaining and direct property access to control raster operations.

## Purpose of rioxarray's approach

The examples shown in "Reading/Writing Remote Sensed Images" Chapter demonstrated basic file opening. Rioxarray allows easy control over raster properties like CRS, nodata values, resolution, and spatial bounds through direct method calls and property access.

For instance you might want to check the spatial extent (bounds) of your analysis area. By accessing bounds with rioxarray properties, you can understand your data's spatial coverage and make informed processing decisions.

## How to access raster properties

To access raster properties with rioxarray, use the `rio` accessor on any opened DataArray. For example:

``` python
import rioxarray as rxr

src = rxr.open_rasterio('image.tif')
print(src.rio.crs)        # coordinate reference system
print(src.rio.nodata)     # nodata value
print(src.rio.bounds())   # spatial bounds
print(src.rio.resolution()) # pixel resolution
```

Rioxarray stores raster properties as attributes and methods accessible through the `rio` accessor.

In [1]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Open with rioxarray (no special configuration needed)
src = rxr.open_rasterio(image_path)

# Display basic properties
print('Keyword:', 'crs'.ljust(15), 'Value:', src.rio.crs)
print('Keyword:', 'nodata'.ljust(15), 'Value:', src.rio.nodata)
print('Keyword:', 'shape'.ljust(15), 'Value:', src.shape)
print('Keyword:', 'dtype'.ljust(15), 'Value:', src.dtype)
print('Keyword:', 'resolution'.ljust(15), 'Value:', src.rio.resolution())

Keyword: crs             Value: EPSG:32621
Keyword: nodata          Value: None
Keyword: shape           Value: (3, 1860, 2041)
Keyword: dtype           Value: uint16
Keyword: resolution      Value: (30.0, -30.0)


### Setting Image Properties
Raster properties can be modified using rioxarray methods. This includes setting nodata values for masking operations and applying pixel value scaling via multiplication.

In [2]:
import rioxarray as rxr
import numpy as np

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Set 0 as missing value
src = rxr.open_rasterio(image_path).rio.write_nodata(0)
print("Nodata value:", src.rio.nodata)

# Replace 0 with nan (equivalent to mask_nodata)
src_masked = src.where(src != 0, np.nan)

Nodata value: 0


In [3]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Apply scale factor of 0.0001
src = rxr.open_rasterio(image_path)
src_scaled = src * 0.0001
print("Scale factor applied: 0.0001")

Scale factor applied: 0.0001


### Coordinate Reference System Transformations

To transform the CRS of raster data, use rioxarray's `rio.reproject()` method. This handles on-the-fly coordinate system transformations using rasterio's virtual warping capabilities.

In [4]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
proj4 = "+proj=aea +lat_1=-5 +lat_2=-42 +lat_0=-32 +lon_0=-60 +x_0=0 +y_0=0 +ellps=aust_SA +units=m +no_defs"

# Without transformation
src = rxr.open_rasterio(image_path)
print("Original CRS:", src.rio.crs)

# With CRS transformation
src_transformed = src.rio.reproject(proj4)
print("Transformed CRS:", src_transformed.rio.crs)

Original CRS: EPSG:32621
Transformed CRS: ESRI:102033


### Resolution and CRS Transformation Combined

Multiple transformation parameters can be combined in a single reproject operation. In this example, the raster CRS is transformed from UTM to Albers Equal Area with a resampled cell size of 100m x 100m.

In [5]:
import rioxarray as rxr
from rasterio.enums import Resampling

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
proj4 = "+proj=aea +lat_1=-5 +lat_2=-42 +lat_0=-32 +lon_0=-60 +x_0=0 +y_0=0 +ellps=aust_SA +units=m +no_defs"

# Without transformation
src = rxr.open_rasterio(image_path)
print("Original resolution:", src.rio.resolution())

# With CRS and resolution transformation
src_transformed = src.rio.reproject(proj4, resolution=(100, 100), resampling=Resampling.bilinear)
print("New resolution:", src_transformed.rio.resolution())

Original resolution: (30.0, -30.0)
New resolution: (100.0, -100.0)


### Spatial Subsetting by Bounds

To subset a raster by spatial bounds, use rioxarray's `rio.clip_box()` method. Bounds should be specified as a tuple of (left, bottom, right, top) coordinates in the raster's coordinate system.

In [6]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
bounds = (724634.17, -2806501.39, 737655.48, -2796221.42)

# Without clipping
src = rxr.open_rasterio(image_path)
print("Original bounds:", src.rio.bounds())

# With bounds clipping
src_clipped = src.rio.clip_box(*bounds)
print("Clipped bounds:", src_clipped.rio.bounds())

Original bounds: (717345.0, -2832795.0, 778575.0, -2776995.0)
Clipped bounds: (724605.0, -2806515.0, 737685.0, -2796195.0)


### Raster Alignment and Reference Matching

Rioxarray provides `rio.reproject_match()` to align one raster to match another raster's spatial properties (bounds, CRS, and resolution). This ensures consistent spatial alignment between datasets.

In [7]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
ref_image_path = "../../pygis/data/LC08_L1TP_224077_20200518_20200518_01_RT_B2.TIF"

# Without reference matching
src = rxr.open_rasterio(image_path)
print("Original bounds:", src.rio.bounds())

# Load reference image
ref_img = rxr.open_rasterio(ref_image_path)
print("Reference bounds:", ref_img.rio.bounds())

# Match to reference
src_matched = src.rio.reproject_match(ref_img)
print("Matched bounds:", src_matched.rio.bounds())

Original bounds: (717345.0, -2832795.0, 778575.0, -2776995.0)
Reference bounds: (694005.0, -2812065.0, 754185.0, -2766615.0)
Matched bounds: (694005.0, -2812065.0, 754185.0, -2766615.0)


(f_rs_crs_sensors)=
### Band Naming and Sensor Configurations

Since rasters are opened as DataArrays, bands have coordinate names. By default, bands are numbered starting from 1. However, it's often more intuitive to assign meaningful names corresponding to wavelengths or sensor bands using xarray's `assign_coords()` method.

In [8]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Without band names
src = rxr.open_rasterio(image_path)
print("Original bands:", src.band.values)
print("Number of bands:", len(src.band))

# With BGR band configuration 
if len(src.band) >= 3:
    src_bgr = src.isel(band=[0, 1, 2]).assign_coords(band=['blue', 'green', 'red'])
    print("BGR bands:", src_bgr.band.values)

Original bands: [1 2 3]
Number of bands: 3
BGR bands: ['blue' 'green' 'red']


### Available Sensor Configurations

Different sensors have standard band configurations. Here are the main ones you can implement:

In [9]:
# Available sensor configurations
sensor_configs = {
    'rgb': ['red', 'green', 'blue'],
    'bgr': ['blue', 'green', 'red'], 
    'rgbn': ['red', 'green', 'blue', 'nir'],
    'bgrn': ['blue', 'green', 'red', 'nir'],
    'l5': ['blue', 'green', 'red', 'nir', 'swir1', 'thermal', 'swir2'],
    'l7': ['blue', 'green', 'red', 'nir', 'swir1', 'thermal', 'swir2'],
    'l7th': ['blue', 'green', 'red', 'nir', 'swir1', 'thermal', 'swir2', 'thermal2'],
    'l8': ['coastal', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pan', 'cirrus', 'thermal1', 'thermal2'],
    'l8_basic': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'],
    's2': ['blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir', 'rededge4', 'swir1', 'swir2'],
    'ps': ['blue', 'green', 'red', 'nir'],  # PlanetScope
    'qb': ['blue', 'green', 'red', 'nir'],  # Quickbird
    'ik': ['blue', 'green', 'red', 'nir']   # IKONOS
}

print("Available sensor configurations:")
for sensor_name, bands in sensor_configs.items():
    print(f'{sensor_name.ljust(15)}: {bands}')

Available sensor configurations:
rgb            : ['red', 'green', 'blue']
bgr            : ['blue', 'green', 'red']
rgbn           : ['red', 'green', 'blue', 'nir']
bgrn           : ['blue', 'green', 'red', 'nir']
l5             : ['blue', 'green', 'red', 'nir', 'swir1', 'thermal', 'swir2']
l7             : ['blue', 'green', 'red', 'nir', 'swir1', 'thermal', 'swir2']
l7th           : ['blue', 'green', 'red', 'nir', 'swir1', 'thermal', 'swir2', 'thermal2']
l8             : ['coastal', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pan', 'cirrus', 'thermal1', 'thermal2']
l8_basic       : ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
s2             : ['blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir', 'rededge4', 'swir1', 'swir2']
ps             : ['blue', 'green', 'red', 'nir']
qb             : ['blue', 'green', 'red', 'nir']
ik             : ['blue', 'green', 'red', 'nir']


### Applying Sensor Configurations

You can apply these sensor configurations to your raster data:

In [10]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Open raster
src = rxr.open_rasterio(image_path)
print(f"Available bands: {len(src.band)}")

# Apply BGR configuration
if len(src.band) >= 3:
    band_config = ['blue', 'green', 'red']
    src_bgr = src.isel(band=[0, 1, 2]).assign_coords(band=band_config)
    print("Applied BGR configuration:", src_bgr.band.values)

# Apply L8 basic configuration if enough bands
if len(src.band) >= 6:
    band_config = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
    src_l8 = src.isel(band=list(range(6))).assign_coords(band=band_config)
    print("Applied L8 basic configuration:", src_l8.band.values)

Available bands: 3
Applied BGR configuration: ['blue' 'green' 'red']
